In [4]:
# Import libraries
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import pandas as pd
import threading
import time

class TradeApp(EWrapper, EClient): 
    def __init__(self): 
        EClient.__init__(self, self) 
        self.data = {}
        
    def historicalData(self, reqId, bar):
        if reqId not in self.data:
            self.data[reqId] = pd.DataFrame([{"Date":bar.date,
                                              "Open":bar.open,
                                              "High":bar.high,
                                              "Low":bar.low,
                                              "Close":bar.close,
                                              "Volume":bar.volume}])
        else:
            self.data[reqId] = pd.concat((self.data[reqId],
                                          pd.DataFrame([{"Date":bar.date,
                                                         "Open":bar.open,
                                                         "High":bar.high,
                                                         "Low":bar.low,
                                                         "Close":bar.close,
                                                         "Volume":bar.volume}])))
            #self.data[reqId].append({"Date":bar.date,"Open":bar.open,"High":bar.high,"Low":bar.low,"Close":bar.close,"Volume":bar.volume})
        print("reqID:{}, date:{}, open:{}, high:{}, low:{}, close:{}, volume:{}".
            format(reqId,bar.date,bar.open,bar.high,bar.low,bar.close,bar.volume))
        
    def historicalDataEnd(self, reqId, start, end):
        super().historicalDataEnd(reqId, start, end)
        print("HistoricalDataEnd. ReqId:", reqId, "from", start, "to", end)
        event.set()

def usTechStk(symbol,sec_type="STK",currency="USD",exchange="ISLAND"):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = sec_type
    contract.currency = currency
    contract.exchange = exchange
    return contract 

def histData(req_num,contract,duration,candle_size):
    """extracts historical data"""
    app.reqHistoricalData(reqId=req_num, 
                          contract=contract,
                          endDateTime='',
                          durationStr=duration,
                          barSizeSetting=candle_size,
                          whatToShow='ADJUSTED_LAST',
                          useRTH=1,
                          formatDate=1,
                          keepUpToDate=0,
                          chartOptions=[])	 # EClient function to request contract details

def websocket_con():
    app.run()
    
app = TradeApp()
app.connect(host='127.0.0.1', port=7497, clientId=33) #port 4002 for ib gateway paper trading/7497 for TWS paper trading
con_thread = threading.Thread(target=websocket_con, daemon=True)
con_thread.start()
time.sleep(1) # some latency added to ensure that the connection is established

event = threading.Event()

tickers = ["TSLA","MSFT","ORCL"]
for ticker in tickers:
    event.clear()
    histData(tickers.index(ticker),usTechStk(ticker),'1 D', '10 secs')
    event.wait()
    
###################storing trade app object in dataframe#######################
def dataDataframe(symbols,TradeApp_obj):
    "returns extracted historical data in dataframe format"
    df_data = {}
    for symbol in symbols:
        df_data[symbol] = pd.DataFrame(TradeApp_obj.data[symbols.index(symbol)])
        df_data[symbol].set_index("Date",inplace=True)
    return df_data

#extract and store historical data in dataframe
historicalData = dataDataframe(tickers,app)

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:afarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:apachmds
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj
ERROR 0 2176 Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 4440.03 to 4440


reqID:0, date:20250627  20:30:00, open:324.55, high:324.57, low:322.18, close:322.18, volume:4440
reqID:0, date:20250627  20:30:10, open:322.29, high:322.35, low:321.2, close:321.34, volume:450
reqID:0, date:20250627  20:30:20, open:321.26, high:321.37, low:320.6, close:321.34, volume:429
reqID:0, date:20250627  20:30:30, open:321.24, high:321.88, low:321.22, close:321.5, volume:183
reqID:0, date:20250627  20:30:40, open:321.54, high:322.21, low:321.44, close:322.17, volume:136
reqID:0, date:20250627  20:30:50, open:322.18, high:322.31, low:321.93, close:322.22, volume:69
reqID:0, date:20250627  20:31:00, open:322.25, high:323.0, low:322.17, close:322.5, volume:120
reqID:0, date:20250627  20:31:10, open:322.48, high:323.0, low:322.48, close:322.48, volume:153
reqID:0, date:20250627  20:31:20, open:322.4, high:322.4, low:321.31, close:321.63, volume:157
reqID:0, date:20250627  20:31:30, open:321.63, high:321.75, low:320.88, close:321.67, volume:132
reqID:0, date:20250627  20:31:40, open

ERROR 1 2176 Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 4989.01 to 4989


reqID:1, date:20250627  20:30:00, open:497.54, high:498.1, low:497.17, close:497.9, volume:4989
reqID:1, date:20250627  20:30:10, open:497.81, high:497.83, low:497.41, close:497.58, volume:21
reqID:1, date:20250627  20:30:20, open:497.31, high:497.49, low:497.26, close:497.49, volume:6
reqID:1, date:20250627  20:30:30, open:497.54, high:498.07, low:497.46, close:498.07, volume:23
reqID:1, date:20250627  20:30:40, open:498.06, high:498.06, low:497.95, close:497.95, volume:2
reqID:1, date:20250627  20:30:50, open:498.0, high:498.08, low:498.0, close:498.08, volume:9
reqID:1, date:20250627  20:31:00, open:498.09, high:498.09, low:497.8, close:497.98, volume:29
reqID:1, date:20250627  20:31:10, open:497.99, high:497.99, low:497.96, close:497.96, volume:2
reqID:1, date:20250627  20:31:20, open:498.05, high:498.07, low:497.9, close:498.04, volume:14
reqID:1, date:20250627  20:31:30, open:498.01, high:498.08, low:498.01, close:498.07, volume:6
reqID:1, date:20250627  20:31:40, open:498.08, hi

ERROR 2 2176 Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 4.35 to 4


reqID:2, date:20250627  20:30:00, open:213.84, high:213.84, low:213.7, close:213.7, volume:4
reqID:2, date:20250627  20:30:10, open:214.0, high:214.0, low:214.0, close:214.0, volume:1
reqID:2, date:20250627  20:30:20, open:213.91, high:213.93, low:213.87, close:213.87, volume:5
reqID:2, date:20250627  20:30:30, open:213.98, high:214.09, low:213.96, close:214.09, volume:4
reqID:2, date:20250627  20:30:40, open:214.09, high:214.09, low:214.09, close:214.09, volume:0
reqID:2, date:20250627  20:30:50, open:214.0, high:214.0, low:213.99, close:214.0, volume:1
reqID:2, date:20250627  20:31:00, open:214.0, high:214.0, low:213.99, close:214.0, volume:3
reqID:2, date:20250627  20:31:10, open:214.0, high:214.0, low:213.99, close:214.0, volume:0
reqID:2, date:20250627  20:31:20, open:213.99, high:214.02, low:213.71, close:214.0, volume:36
reqID:2, date:20250627  20:31:30, open:213.9, high:214.03, low:213.9, close:214.03, volume:17
reqID:2, date:20250627  20:31:40, open:213.89, high:214.09, low:21

In [5]:
app.disconnect()

In [6]:
historicalData

{'TSLA':                       Open    High     Low   Close  Volume
 Date                                                      
 20250627  20:30:00  324.55  324.57  322.18  322.18    4440
 20250627  20:30:10  322.29  322.35  321.20  321.34     450
 20250627  20:30:20  321.26  321.37  320.60  321.34     429
 20250627  20:30:30  321.24  321.88  321.22  321.50     183
 20250627  20:30:40  321.54  322.21  321.44  322.17     136
 ...                    ...     ...     ...     ...     ...
 20250628  02:59:10  323.62  323.77  323.56  323.75     361
 20250628  02:59:20  323.73  323.78  323.60  323.64     225
 20250628  02:59:30  323.65  323.75  323.58  323.58     326
 20250628  02:59:40  323.59  323.81  323.59  323.77     582
 20250628  02:59:50  323.75  323.88  323.60  323.80    1188
 
 [2340 rows x 5 columns],
 'MSFT':                       Open    High     Low   Close  Volume
 Date                                                      
 20250627  20:30:00  497.54  498.10  497.17  497.90    4

In [4]:
app.disconnect()
import pickle

with open('META_AMZN_VOO_200D_15min.pkl', 'wb') as f:
    pickle.dump(historicalData, f)

In [5]:
historicalData

{'META':                       Open    High     Low   Close  Volume
 Date                                                      
 20250530  20:30:00  642.92  643.54  638.65  640.28    1767
 20250530  20:45:00  640.41  641.62  638.72  641.30     375
 20250530  21:00:00  641.76  645.90  641.76  645.76     430
 20250530  21:15:00  645.50  646.15  644.71  644.95     198
 20250530  21:30:00  644.77  646.53  644.41  645.05     225
 ...                    ...     ...     ...     ...     ...
 20250628  01:45:00  727.60  728.85  725.91  726.70     314
 20250628  02:00:00  726.85  728.94  726.85  728.94     166
 20250628  02:15:00  728.82  730.00  728.56  728.98     239
 20250628  02:30:00  729.20  730.09  729.12  730.09     213
 20250628  02:45:00  729.94  733.91  728.31  733.69    2987
 
 [520 rows x 5 columns],
 'AMZN':                       Open    High     Low   Close  Volume
 Date                                                      
 20250530  20:30:00  204.89  205.25  203.96  203.98    60